In [1]:
import pickle
import numpy as np
from dataset import Dataset
import plotly.graph_objects as go

In [2]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

dataset = Dataset(sensor_data, labels, interp_funcs)

In [3]:
def plot_data_pair(X_l, X_r, time_arr, title):
    data_T_l = X_l.T
    data_T_r = X_r.T
    fig = go.Figure()

    for i, data in enumerate(data_T_l):
        fig.add_trace(go.Scatter(x=time_arr,
                                 y=data,
                                 mode="markers",
                                 name=f"L {i}"))
    for i, data in enumerate(data_T_r):
        fig.add_trace(go.Scatter(x=time_arr,
                                 y=data,
                                 mode="markers",
                                 name=f"R {i}"))

    fig.update_layout(title=title, title_x=0.5, width=1000, height=600)
    fig.update_traces(marker=dict(size=2))
    fig.show()

In [4]:
X, y, time_array, _ = dataset.get_sensor_pair_cls(1,
                                                  (0, 1),
                                                  num_samples=100,
                                                  as_log=True,
                                                  as_mean=False)

X_src, X_target = X[:, :10], X[:, 10:]
X_src_calibrated = dataset.calibrate_data(X_src, X_target)
plot_data_pair(X_src_calibrated, X_target, time_array, "")

In [5]:
X, y, time_array, _ = dataset.get_sensor_pair_cls(0,
                                                  (0, 1),
                                                  num_samples=100,
                                                  as_log=False,
                                                  as_mean=False)
X_l, X_r = X[:, :10], X[:, 10:]
errors = []
for i in range(len(X_l)):
    l_err = np.mean(np.abs((X_l[i] - X_r[i])) / X_l[i])
    r_err = np.mean(np.abs((X_l[i] - X_r[i])) / X_r[i])
    errors.append(min(l_err, r_err))

errors = np.array(errors)
print(errors.mean())

0.25717915481865333
